# MUSIC RECOMMENDER FINAL STEP

Aplying the recommender model to our top 100 and big playlist.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import cluster, datasets

In [2]:
#Open the top 100

In [3]:
Top_100 = pd.read_csv('music_top100.csv')
Top_100.head()

,title,artist
0,"81 Million Votes, My Ass",The Truth Bombers & Kari Lake
1,Fast Car,Luke Combs
2,Last Night,Morgan Wallen
3,Take Two,BTS
4,Need A Favor,Jelly Roll


In [4]:
#Open the final_playlist

In [5]:
Big_playlist = pd.read_csv('final_playlist.csv')
Big_playlist.head()

,title,artist,cluster
0,You're The One - Un mal pour un bien,Petula Clark,0
1,Doctor My Eyes,Jackson Browne,4
2,Falling in Love at a Coffee Shop,Landon Pigg,7
3,Peaceful Easy Feeling - 2013 Remaster,Eagles,4
4,Summertime Blues,Eddie Cochran,3


In [6]:
import pickle

kmeans = pickle.load(open('kmeans.pkl', 'rb'))
scaler = pickle.load(open('scaler.pkl', 'rb'))

In [7]:
kmeans

KMeans()

In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [9]:
secrets_file = open("secrets.txt","r")

In [10]:
string = secrets_file.read()

In [11]:
#string.split('\n')

In [12]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [13]:
#secrets_dict

In [14]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['ClientID'],
                                                           client_secret=secrets_dict['Clientsecret']))

# Looking for Input audio features

In [15]:
#user_input = input("Type a song title: ")

In [16]:
#song = sp.search(q=user_input, type='track', limit=1)

In [17]:
#song_uri = song['tracks']['items'][0]['uri']

In [18]:
#song_features = sp.audio_features(song_uri)
#song_features

In [19]:
#song_features_df = pd.DataFrame(song_features)
#song_features_df

In [20]:
#song_features_df1 = song_features_df[['danceability', 'energy', 'key', 'loudness', 'mode','speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence','tempo', 'duration_ms']]
#song_features_df1

In [21]:
#song_features_scaled = StandardScaler().fit_transform(song_features_df1)
#song_features_scaled

In [22]:
#cluster1 = kmeans.predict(song_features_scaled)
#cluster1

# Functions for recommendation

In [23]:
def song_features(user_input):
        song = sp.search(q=user_input, type='track', limit=1)
        song_uri = song['tracks']['items'][0]['uri']
        song_feat = sp.audio_features(song_uri)
        return song_feat
    
user_input = input("Type a song title: ")
song_feat = song_features(user_input)

print(song_feat)

Type a song title: fast car
[{'danceability': 0.712, 'energy': 0.603, 'key': 8, 'loudness': -5.52, 'mode': 1, 'speechiness': 0.0262, 'acousticness': 0.186, 'instrumentalness': 0, 'liveness': 0.115, 'valence': 0.67, 'tempo': 97.994, 'type': 'audio_features', 'id': '1Lo0QY9cvc8sUB2vnIOxDT', 'uri': 'spotify:track:1Lo0QY9cvc8sUB2vnIOxDT', 'track_href': 'https://api.spotify.com/v1/tracks/1Lo0QY9cvc8sUB2vnIOxDT', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1Lo0QY9cvc8sUB2vnIOxDT', 'duration_ms': 265493, 'time_signature': 4}]


In [24]:
def song_df(song_feat):
    song_features_df = pd.DataFrame(song_feat)
    song_features_df1 = song_features_df[['danceability', 'energy', 'key', 'loudness', 'mode','speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence','tempo', 'duration_ms']]
    return song_features_df1

song_features_df1 = song_df(song_feat)
song_features_df1

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.712,0.603,8,-5.52,1,0.0262,0.186,0,0.115,0.67,97.994,265493


In [25]:
def song_cluster(song_features_df1):
    song_features_scaled = scaler.transform(song_features_df1)
    cluster = kmeans.predict(song_features_scaled)
    return cluster 

cluster = list(song_cluster(song_features_df1))[0]
cluster


C:\Users\Andrea\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


4

# Music Recommender

In [26]:
#pip install fuzzywuzzy

In [30]:
from fuzzywuzzy import fuzz

def recommend_song(user_input, Top_100, Big_playlist):
    if any(fuzz.ratio(user_input.lower(), title.lower()) >= 90 for title in Top_100['title'].values):
        # I the song is in the Top 100
        # Select a ramdom song from Top 100
        random_song = Top_100[Top_100['title'].apply(lambda x: fuzz.ratio(user_input.lower(), x.lower()) < 90)].sample(1)
        recommended_song = random_song.iloc[0]['title']
        recommended_artist = random_song.iloc[0]['artist']
        return f"Fantastic, your song is in the Top 100; here is another suggestion: '{recommended_song}' by {recommended_artist}, enjoy!"
    
    else:
        # If the song is not in the Top 100
        # Get the features of the song and apply a cluster for it and recommend
        song_feat = song_features(user_input)
        if len(song_feat) == 0:
            return "Sorry, I don't have a recommendation at this time. Try with a new song."
        else:
            song_features_df1 = song_df(song_feat)
            cluster = list(song_cluster(song_features_df1))[0]

            # Select a ramdom song from Big_playlist with the same cluster
            cluster_songs = Big_playlist[Big_playlist['cluster'] == cluster]
            random_song = cluster_songs.sample(1)
            recommended_song = random_song.iloc[0]['title']
            recommended_artist = random_song.iloc[0]['artist']
            return f"Great! I recommend you listen to: '{recommended_song}' by {recommended_artist}, enjoy!"

# User input for the song
user_input = input("Type a song title: ").lower()

recommendation = recommend_song(user_input, Top_100, Big_playlist)
print(recommendation)


Type a song title: fast car
Fantastic, your song is in the Top 100; here is another suggestion: '81 Million Votes, My Ass' by The Truth Bombers & Kari Lake, enjoy!


# Verify that the song is in the Top 100.

In [29]:
def check_song_in_top100(song_title, top100_df):
    song_exists = Top_100['title'].str.contains(song_title, case=False).any()
    return song_exists

song_title = "gasolina"  

if check_song_in_top100(song_title, Top_100):
    print("The song is in the Top 100.")
else:
    print("The song is not in the Top 100")


The song is not in the Top 100


# Verify that the song is in the Big playlist.

In [ ]:
def check_song_in_big_playlist(song_title, top100_df):
    song_exists = Big_playlist['title'].str.contains(song_title, case=False).any()
    return song_exists

song_title = "Doctor My Eyes"  

if check_song_in_big_playlist(song_title, Big_playlist):
    print("The song is in the Big playlist.")
else:
    print("The song is not in the Big playlist")
